In [7]:
import pymongo
import pandas as pd
import json
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import datetime


# Connexion à MongoDB
cluster = pymongo.MongoClient("mongodb://localhost:27017/")
db = cluster["ProjectBigData"]

# Collection "topVideo"
collection = db['topVideo']

# Chargement des données depuis le fichier CSV "dailyTop200.csv"
df = pd.read_csv('dailyTop200.csv')
# Conversion du DataFrame en dictionnaire pour l'insertion dans la base de données
data_dict = df.to_dict(orient='records')
collection.insert_many(data_dict)

# Collection "comments"
collection = db['comments']
# Création de l'index unique sur le champ "id" dans la collection "comments"
collection.create_index([("id", pymongo.ASCENDING), ("videoId", pymongo.ASCENDING)], unique=True)
# Chargement des données depuis le fichier CSV "comments.csv"
df = pd.read_csv('comments.csv')
data_dict = df.to_dict(orient='records')
for document in data_dict:
    try:
        collection.insert_one(document)
    except pymongo.errors.DuplicateKeyError as e:
       
        continue

C:\Users\jerem\AppData\Local\Temp\ipykernel_12716\2345366561.py:27: DtypeWarning: Columns (0,3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('comments.csv')


Ignoré document en raison d'une clé dupliquée : {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: ProjectBigData.comments index: id_1_videoId_1 dup key: { id: "UgxHYBQAE58AMk_Njg14AaABAg", videoId: null }', 'keyPattern': {'id': 1, 'videoId': 1}, 'keyValue': {'id': 'UgxHYBQAE58AMk_Njg14AaABAg', 'videoId': None}}
Ignoré document en raison d'une clé dupliquée : {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: ProjectBigData.comments index: id_1_videoId_1 dup key: { id: "UgyP3F15VnXEp_W84Z94AaABAg", videoId: null }', 'keyPattern': {'id': 1, 'videoId': 1}, 'keyValue': {'id': 'UgyP3F15VnXEp_W84Z94AaABAg', 'videoId': None}}
Ignoré document en raison d'une clé dupliquée : {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: ProjectBigData.comments index: id_1_videoId_1 dup key: { id: "Ugygqdx4KLS--EiHxch4AaABAg", videoId: null }', 'keyPattern': {'id': 1, 'videoId': 1}, 'keyValue': {'id': 'Ugygqdx4KLS--EiHxch4AaAB